In [1]:
import pandas as pd
dataset=pd.read_csv("Placement.csv")
dataset

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [2]:
dataset.columns

Index(['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s',
       'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p',
       'status', 'salary'],
      dtype='object')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sl_no           215 non-null    int64  
 1   gender          215 non-null    object 
 2   ssc_p           215 non-null    float64
 3   ssc_b           215 non-null    object 
 4   hsc_p           215 non-null    float64
 5   hsc_b           215 non-null    object 
 6   hsc_s           215 non-null    object 
 7   degree_p        215 non-null    float64
 8   degree_t        215 non-null    object 
 9   workex          215 non-null    object 
 10  etest_p         215 non-null    float64
 11  specialisation  215 non-null    object 
 12  mba_p           215 non-null    float64
 13  status          215 non-null    object 
 14  salary          148 non-null    float64
dtypes: float64(6), int64(1), object(8)
memory usage: 25.3+ KB


In [11]:
quan= dataset.select_dtypes(include=["number"]).columns.tolist()
qual= dataset.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
print(quan)
print(qual)

['sl_no', 'ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']
['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']


In [24]:
def quanqual(dataset):
    quan=[]
    qual=[]
    for columnname in dataset.columns:
        if (dataset[columnname].dtype=='o'):
            qual.append(columnname)
        else:
            quan.append(columnname)
    return quan,qual
# quan, qual = quanqual(dataset)
# print("Quantitative:", quan)
# print("Qualitative:", qual)


In [26]:
def quanqual(dataset):
    quan = dataset.select_dtypes(include=["number"]).columns.tolist()
    qual = dataset.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
    return quan, qual
    
quan,qual =quanqual(dataset)
print("Quantitative:", quan)
print("Qualitative:", qual)


Quantitative: ['sl_no', 'ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']
Qualitative: ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']


In [4]:
import pandas as pd
df=pd.read_csv("placement_reorder.csv")

def quanqual(df, id_cols=None, treat_bool_as_cat=True, round_digits=2, to_csv=None):
    """
    Split quantitative/qualitative columns and return a compact stats table.

    Parameters
    ----------
    df : pd.DataFrame
    id_cols : list[str] | None
        Columns to ignore as features (e.g., ['sl_no']).
    treat_bool_as_cat : bool
        If True, booleans go to qualitative; if False they’re treated as numeric.
    round_digits : int
        Rounding for numeric summary.
    to_csv : str | None
        If provided, saves the stats table to this CSV path.

    Returns
    -------
    quant_cols : list[str]
    qual_cols  : list[str]
    stats_df   : pd.DataFrame  # columns: ['Numerical Column Name','Mean','Median','Mode']
    """
    id_cols = id_cols or []
    df_ = df.drop(columns=id_cols, errors="ignore")

    cat_types = ["object", "category"] + (["bool"] if treat_bool_as_cat else [])
    quant_cols = df_.select_dtypes(include=["number"]).columns.tolist()
    qual_cols  = df_.select_dtypes(include=cat_types).columns.tolist()

    if quant_cols:
        # Vectorized summary
        stats = df_[quant_cols].agg(["mean", "median"]).T
        # First mode per column (handles ties by taking the first)
        modes = df_[quant_cols].mode(dropna=True)
        if not modes.empty:
            stats["Mode"] = modes.iloc[0]
        else:
            stats["Mode"] = pd.NA
        stats = stats.round(round_digits)
        stats.index.name = "Numerical Column Name"
        stats_df = stats.reset_index()[["Numerical Column Name", "mean", "median", "Mode"]]
        stats_df = stats_df.rename(columns={"mean": "Mean", "median": "Median"})
    else:
        stats_df = pd.DataFrame(columns=["Numerical Column Name", "Mean", "Median", "Mode"])

    if to_csv:
        stats_df.to_csv(to_csv, index=False)

    return quant_cols, qual_cols, stats_df
quant, qual, stats_table = quanqual(df, id_cols=["sl_no"], to_csv="numerical_summary.csv")